In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import datetime
import gc

In [2]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
df_merge = reduce_mem_usage(pd.read_csv('./output/outlier_remove.csv'))
target = df_merge.logerror
features = df_merge.drop(['logerror'], axis = 1)

Memory usage of dataframe is 58.98 MB
Memory usage after optimization is: 23.37 MB
Decreased by 60.4%


In [4]:
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization
categorical_features = ['airconditioningtypeid', 'hashottuborspa', 'heatingorsystemtypeid', 
                       'pooltypeid2', 'propertylandusetypeid', 'fips', 'regionidcounty', 
                       'buildingqualitytypeid_fill', 'regionidcity_fill', 'year', 
                       'regionidneighborhood_fill', 'taxdelinquencyflag']

In [7]:
params = {"objective": "regression", "boosting": "gbdt", "num_leaves": 512, 
              "learning_rate": 0.0021, 'bagging_fraction': 0.85, "reg_lambda": 0.1, 
              'reg_alpha':0.1, "metric": "rmse", 'max_depth': -1, 'min_child_weight': 30,
              'verbose': -1, 'min_split_gain':0.1, 'subsample_freq':1, 'sub_feature':  0.5}

In [6]:
def bayes_parameter_opt_lgb(X, y, init_round=3, opt_round=20, n_folds=3, 
                            random_seed=6, n_estimators=3000):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y,categorical_feature=categorical_features,free_raw_data=False)
    # parameters
    def lgb_eval(num_leaves, learning_rate, bagging_fraction, reg_lambda, reg_alpha, max_depth,
                 min_child_weight, min_split_gain, subsample_freq, sub_feature):
        params = {'objective':'regression', 'boosting_type': 'gbdt', 'verbose': -1, 
                  'num_boost_round': n_estimators, "metric": "rmse"}
        params["num_leaves"] = int(round(num_leaves))
        params['learning_rate'] = learning_rate
        params['bagging_fraction'] = bagging_fraction
        params['reg_lambda'] = max(reg_lambda, 0)
        params['reg_alpha'] = max(reg_alpha, 0)
        params['max_depth'] = int(round(max_depth))
        params['min_child_weight'] = min_child_weight
        params['min_split_gain'] = min_split_gain
        params['subsample_freq']=int(round(subsample_freq))
        params['sub_feature'] = sub_feature      
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=False, 
                           verbose_eval=25, metrics=['rmse'],early_stopping_rounds=50)
        return -1.0 * np.min(cv_result['rmse-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (500,700),
                                            'learning_rate': (0.001,0.003),
                                            'bagging_fraction': (0.8, 0.9),
                                            "reg_lambda": (0.07, 0.13),
                                            'reg_alpha':(0.08, 0.12),
                                            'max_depth': (-1, 10),
                                            'min_child_weight': (27, 33),
                                            'min_split_gain':(0.08, 0.12),
                                            'subsample_freq':(0.8, 1.2),
                                            'sub_feature':  (0.4,0.6)})
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)

In [8]:
import warnings
opt_params = bayes_parameter_opt_lgb(features, target, init_round=3, opt_round=15, 
                                     n_folds=3, random_seed=6, n_estimators=3000)

|   iter    |  target   | baggin... | learni... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | sub_fe... | subsam... |
-------------------------------------------------------------------------------------------------------------------------------------------------


/local/home/ningzesun/.local/lib/python3.6/site-packages/lightgbm/engine.py:503: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/local/home/ningzesun/.local/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[25]	cv_agg's rmse: 0.0830788 + 0.00018015
[50]	cv_agg's rmse: 0.0830512 + 0.000180667
[75]	cv_agg's rmse: 0.0830238 + 0.000181203
[100]	cv_agg's rmse: 0.0829989 + 0.000181677
[125]	cv_agg's rmse: 0.0829723 + 0.000182325
[150]	cv_agg's rmse: 0.0829457 + 0.000182707
[175]	cv_agg's rmse: 0.0829236 + 0.000183337
[200]	cv_agg's rmse: 0.0829006 + 0.000184139
[225]	cv_agg's rmse: 0.082878 + 0.000185204
[250]	cv_agg's rmse: 0.0828576 + 0.000185805
[275]	cv_agg's rmse: 0.0828378 + 0.000186252
[300]	cv_agg's rmse: 0.082818 + 0.000186947
[325]	cv_agg's rmse: 0.0827993 + 0.000187891
[350]	cv_agg's rmse: 0.0827809 + 0.000188537
[375]	cv_agg's rmse: 0.0827647 + 0.000188724
[400]	cv_agg's rmse: 0.0827479 + 0.000189325
[425]	cv_agg's rmse: 0.0827326 + 0.000189872
[450]	cv_agg's rmse: 0.0827157 + 0.00019061
[475]	cv_agg's rmse: 0.0827008 + 0.000191288
[500]	cv_agg's rmse: 0.0826873 + 0.000191763
[525]	cv_agg's rmse: 0.082673 + 0.00019226
[550]	cv_agg's rmse: 0.0826586 + 0.000192632
[575]	cv_agg's rmse

[1425]	cv_agg's rmse: 0.0823974 + 0.00020663
[1450]	cv_agg's rmse: 0.0823931 + 0.000206466
[1475]	cv_agg's rmse: 0.0823881 + 0.000206677
[1500]	cv_agg's rmse: 0.0823829 + 0.000206849
[1525]	cv_agg's rmse: 0.0823781 + 0.000206895
[1550]	cv_agg's rmse: 0.0823742 + 0.000207228
[1575]	cv_agg's rmse: 0.0823693 + 0.000207583
[1600]	cv_agg's rmse: 0.0823652 + 0.000207625
[1625]	cv_agg's rmse: 0.0823605 + 0.000207868
[1650]	cv_agg's rmse: 0.0823565 + 0.00020812
[1675]	cv_agg's rmse: 0.0823517 + 0.000208016
[1700]	cv_agg's rmse: 0.0823478 + 0.000208251
[1725]	cv_agg's rmse: 0.0823439 + 0.000207984
[1750]	cv_agg's rmse: 0.0823404 + 0.000208162
[1775]	cv_agg's rmse: 0.0823366 + 0.00020824
[1800]	cv_agg's rmse: 0.082333 + 0.000208311
[1825]	cv_agg's rmse: 0.0823294 + 0.000208306
[1850]	cv_agg's rmse: 0.0823257 + 0.000208505
[1875]	cv_agg's rmse: 0.0823222 + 0.000208566
[1900]	cv_agg's rmse: 0.0823192 + 0.00020885
[1925]	cv_agg's rmse: 0.0823162 + 0.000208643
[1950]	cv_agg's rmse: 0.0823124 + 0.000

[2825]	cv_agg's rmse: 0.0823567 + 0.000209376
[2850]	cv_agg's rmse: 0.0823546 + 0.000209056
[2875]	cv_agg's rmse: 0.0823524 + 0.000208939
[2900]	cv_agg's rmse: 0.0823504 + 0.000208924
[2925]	cv_agg's rmse: 0.0823481 + 0.000209018
[2950]	cv_agg's rmse: 0.0823462 + 0.000209038
[2975]	cv_agg's rmse: 0.0823442 + 0.000208812
[3000]	cv_agg's rmse: 0.0823421 + 0.000208896
|  3        | -0.08234  |  0.8466   |  0.001676 |  2.975    |  28.15    |  0.1113   |  518.7    |  0.1033   |  0.09861  |  0.4559   |  1.158    |


...

[25]	cv_agg's rmse: 0.0830266 + 0.000181532
[50]	cv_agg's rmse: 0.0829535 + 0.000183864
[75]	cv_agg's rmse: 0.0828865 + 0.000185818
[100]	cv_agg's rmse: 0.0828258 + 0.000187282
[125]	cv_agg's rmse: 0.0827696 + 0.000189663
[150]	cv_agg's rmse: 0.0827178 + 0.000191508
[175]	cv_agg's rmse: 0.0826749 + 0.000192756
[200]	cv_agg's rmse: 0.0826321 + 0.000194474
[225]	cv_agg's rmse: 0.0825933 + 0.000195438
[250]	cv_agg's rmse: 0.0825587 + 0.000197189
[275]	cv_agg's rmse: 0.0825271 + 0.00019797
[300]	cv_agg's rmse: 0.0824979 + 0.000199855
[325]	cv_agg's rmse: 0.0824718 + 0.0002009
[350]	cv_agg's rmse: 0.0824459 + 0.000201865
[375]	cv_agg's rmse: 0.0824238 + 0.000202965
[400]	cv_agg's rmse: 0.082404 + 0.000204177
[425]	cv_agg's rmse: 0.0823844 + 0.000205462
[450]	cv_agg's rmse: 0.0823673 + 0.000206348
[475]	cv_agg's rmse: 0.082351 + 0.000207578
[500]	cv_agg's rmse: 0.082336 + 0.000208536
[525]	cv_agg's rmse: 0.0823212 + 0.000209516
[550]	cv_agg's rmse: 0.0823078 + 0.000210836
[575]	cv_agg's rmse

/local/home/ningzesun/.local/lib/python3.6/site-packages/lightgbm/engine.py:503: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/local/home/ningzesun/.local/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[25]	cv_agg's rmse: 0.0830608 + 0.000180502
[50]	cv_agg's rmse: 0.0830175 + 0.000181617
[75]	cv_agg's rmse: 0.082974 + 0.000182576
[100]	cv_agg's rmse: 0.0829348 + 0.000183435
[125]	cv_agg's rmse: 0.0828937 + 0.000184454
[150]	cv_agg's rmse: 0.0828534 + 0.000185634
[175]	cv_agg's rmse: 0.0828219 + 0.000186631
[200]	cv_agg's rmse: 0.0827878 + 0.000187607
[225]	cv_agg's rmse: 0.0827545 + 0.000188696
[250]	cv_agg's rmse: 0.0827265 + 0.000189528
[275]	cv_agg's rmse: 0.0827 + 0.000190336
[300]	cv_agg's rmse: 0.0826731 + 0.00019106
[325]	cv_agg's rmse: 0.082648 + 0.000191965
[350]	cv_agg's rmse: 0.0826238 + 0.000192682
[375]	cv_agg's rmse: 0.082603 + 0.000193313
[400]	cv_agg's rmse: 0.0825814 + 0.000194103
[425]	cv_agg's rmse: 0.0825616 + 0.000195156
[450]	cv_agg's rmse: 0.0825412 + 0.00019644
[475]	cv_agg's rmse: 0.0825241 + 0.000197095
[500]	cv_agg's rmse: 0.0825082 + 0.000197534
[525]	cv_agg's rmse: 0.082491 + 0.000197729
[550]	cv_agg's rmse: 0.0824741 + 0.000198573
[575]	cv_agg's rmse: 0

/local/home/ningzesun/.local/lib/python3.6/site-packages/lightgbm/engine.py:503: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/local/home/ningzesun/.local/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[25]	cv_agg's rmse: 0.0830803 + 0.000180044
[50]	cv_agg's rmse: 0.0830528 + 0.000180554
[75]	cv_agg's rmse: 0.0830262 + 0.000181203
[100]	cv_agg's rmse: 0.0830012 + 0.000181739
[125]	cv_agg's rmse: 0.0829746 + 0.000182494
[150]	cv_agg's rmse: 0.082948 + 0.000183261
[175]	cv_agg's rmse: 0.0829265 + 0.000183747
[200]	cv_agg's rmse: 0.0829035 + 0.000184554
[225]	cv_agg's rmse: 0.0828801 + 0.000185287
[250]	cv_agg's rmse: 0.0828594 + 0.000185799
[275]	cv_agg's rmse: 0.0828395 + 0.000186288
[300]	cv_agg's rmse: 0.0828193 + 0.000186812
[325]	cv_agg's rmse: 0.0827998 + 0.000187549
[350]	cv_agg's rmse: 0.0827809 + 0.000188033
[375]	cv_agg's rmse: 0.0827638 + 0.000188227
[400]	cv_agg's rmse: 0.0827463 + 0.00018894
[425]	cv_agg's rmse: 0.0827301 + 0.000189284
[450]	cv_agg's rmse: 0.0827132 + 0.000190132
[475]	cv_agg's rmse: 0.0826981 + 0.000190625
[500]	cv_agg's rmse: 0.0826843 + 0.000190835
[525]	cv_agg's rmse: 0.0826696 + 0.000191411
[550]	cv_agg's rmse: 0.0826546 + 0.000192112
[575]	cv_agg's 

In [ ]:
|   iter    |  target   | baggin... | learni... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | sub_fe... | subsam... |
|  9        | -0.0821   |  0.8297   |  0.002515 |  9.549    |  27.19    |  0.08138  |  500.2    |  0.1046   |  0.1052   |  0.4492   |  0.9387   |
params = {"objective": "regression", "boosting": "gbdt", "num_leaves": 500, 
              "learning_rate": 0.002515, 'bagging_fraction': 0.8297, "reg_lambda": 0.1052, 
              'reg_alpha':0.1046, "metric": "rmse", 'max_depth': 10, 'min_child_weight': 27,
              'verbose': -1, 'min_split_gain':0.08138 , 'subsample_freq':0.9387, 'sub_feature':  0.4492}